---
title: "Population Based HyperParametric Training"
format:
  html:
    code-fold: false
jupyter: python3
---

# Importing the required libraries

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras as kr

2024-11-25 16:13:04.179983: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Preparing the dataset

In [2]:
mnist = kr.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

train_data = {'X':x_train, 'Y':y_train}
test_data = {'X':x_test, 'Y':y_test}

# Template for Model

In [3]:
model = kr.models.Sequential([
  kr.layers.Input((28,28)),
  kr.layers.Flatten(),
  kr.layers.Dense(128, activation='relu'),
  kr.layers.Dropout(0.2),
  kr.layers.Dense(10)
])

# Creating The Optimizer

In [15]:
class PopulationBasedOptimizer:

    def __init__(self, model, NumberOfAgents, CutOffMeasure, NumberOfLearningIterations, InitialHyperParameter):
        self.listOfAgents = [kr.models.clone_model(model) for _ in range(NumberOfAgents)]
        self.HyperParameters = InitialHyperParameter
        self.NumberOfAgents = NumberOfAgents
        self.NumberOfLearningIterations = NumberOfLearningIterations
        self.CutOffMeasure = CutOffMeasure
        self.Iterations = NumberOfLearningIterations
        for Agent, rate in zip(self.listOfAgents,self.HyperParameters):
            optimizer = kr.optimizers.Adam(learning_rate=rate)
            Agent.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

    def PBTOptimise(self, train_data, test_data):
        for iter in range(self.NumberOfLearningIterations):
            for model in self.listOfAgents:
                model.fit(train_data.get('X'), train_data.get('Y'), epochs=1, batch_size=10)
            current_loss = np.zeros(self.NumberOfAgents)
            for i,model in enumerate(self.listOfAgents):
                loss, accuracy = model.evaluate(test_data.get('X'), test_data.get('Y'), verbose=1)
                current_loss[i] = loss
            Best = np.argmin(current_loss)
            BadIndices = np.argsort(current_loss)[int(self.NumberOfAgents * self.CutOffMeasure):]
            for index in BadIndices:
                Best_weights = self.listOfAgents[Best].get_weights()
                self.listOfAgents[index].set_weights(self.listOfAgents[Best].get_weights() + np.random.normal(0,1, shape = (Best_weights).shape))
                optimizer = kr.optimizers.Adam(learning_rate=self.InitialHyperParameter[Best])
                self.listOfAgents[index].compile(optimizer=optimizer, loss='categoricalcross_entropy', metrics=['accuracy'])

        for model in self.listOfAgents:
            model.fit(train_data.get('X'), train_data.get('Y'), epochs=1)
        current_loss = np.zeros_like(NumberOfAgents)
        for i,model in enumerate(self.listOfAgents):
            current_loss[i], accuracy = model.evaluate(test_data.get('X'), test_data.get('Y'), verbose=1)
            BadIndices = np.argsort(current_loss)[int(self.NumberOfAgents * self.CutOffMeasure):]
        Best = np.argmin(current_loss)
        # Please help
        return self.listOfAgents[Best]

In [16]:
InitialHyperParameter = np.random.normal(0,1,10)
PBT = PopulationBasedOptimizer(model=model, NumberOfAgents=10, CutOffMeasure=0.2, NumberOfLearningIterations=20, InitialHyperParameter=InitialHyperParameter)
best_model = PBT.PBTOptimise(train_data = train_data, test_data = test_data)

6000/6000 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.2085 - loss: 9.8183
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.1091 - loss: 10.9919
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.1426 - loss: 9.6923
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.1399 - loss: 11.0319
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.1089 - loss: 9.7926
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.1397 - loss: 10.1959
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.1489 - loss: 10.7780
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.1163 - loss: 9.1726
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.1379 - loss: 11.1670
6000/6000 ━━━━━━━━━━━━━━━━━━━━ 13s 2ms/step - accuracy: 0.1407 - loss: 11.4980
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1978 - loss: 9.9653
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.1044 - loss: 14.2691
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.0921 - l

NameError: name 'shape' is not defined